ML Pipeline Preparation

Follow the instructions below to help you create your ML pipeline.
1. Import libraries and load data from database.

    Import Python libraries
    Load dataset from database with read_sql_table
    Define feature and target variables X and Y



In [1]:
# import libraries
import numpy as np
import pandas as pd

import sklearn.metrics as skm
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier 
from sklearn.neighbors import KNeighborsClassifier

from sqlalchemy import create_engine
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download(['punkt', 'wordnet'])
import time

[nltk_data] Downloading package punkt to C:\Users\Suramya
[nltk_data]     Pathak\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Suramya
[nltk_data]     Pathak\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterRProject.db')
df = pd.read_sql('SELECT * FROM RESPONSE',engine)
X=df["message"]
y = df.iloc[:,4:]
y['related'].replace(2,1,inplace=True)

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    text=re.sub(r"[^a-zA-Z0-9 ]","",text)
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [10]:


pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

Wall time: 0 ns


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


In [6]:

pipeline.fit(X_train, y_train)

Wall time: 0 ns


C:\ana\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\ana\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\ana\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\ana\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\ana\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                  

In [7]:
y_pred=pipeline.predict(X_test)

labels=np.unique(y_pred)
#confusion_mat=confusion_matrix(y_test, y_pred, labels=labels)


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.


In [8]:
cm=skm.multilabel_confusion_matrix(y_test, y_pred, labels=labels)
print(cm)
print(skm.classification_report(y_test, y_pred))
print("Labels:", labels)
#print("Confusion Matrix:\n", confusion_mat)
accuracy=(y_pred==y_test).mean()
print("Accuracy:", accuracy)

[[[ 760 1294]
  [ 491 6107]]

 [[7070  110]
  [ 878  594]]]
              precision    recall  f1-score   support

           0       0.83      0.93      0.87      6598
           1       0.84      0.40      0.55      1472
           2       0.00      0.00      0.00        38
           3       0.75      0.53      0.62      3545
           4       0.57      0.06      0.11       701
           5       0.60      0.04      0.08       446
           6       0.62      0.04      0.07       226
           7       0.00      0.00      0.00       160
           8       0.57      0.06      0.11       267
           9       0.00      0.00      0.00         0
          10       0.80      0.16      0.26       543
          11       0.80      0.43      0.56       965
          12       0.79      0.23      0.36       775
          13       0.71      0.08      0.14       127
          14       0.45      0.03      0.05       191
          15       0.00      0.00      0.00       104
          16       0.

C:\ana\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ana\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\ana\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
C:\ana\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


Accuracy: related                   0.793689
request                   0.885807
offer                     0.995608
aid_related               0.734512
medical_help              0.920134
medical_products          0.949145
search_and_rescue         0.974226
security                  0.981160
military                  0.969602
child_alone               1.000000
water                     0.944637
food                      0.924757
shelter                   0.925798
clothing                  0.986015
money                     0.977809
missing_people            0.987980
refugees                  0.966019
death                     0.956080
other_aid                 0.867545
infrastructure_related    0.934119
transport                 0.953421
buildings                 0.949838
electricity               0.978849
tools                     0.993874
hospitals                 0.990060
shops                     0.996070
aid_centers               0.986708
other_infrastructure      0.955617
weather_re

In [9]:
accuracy.min()

0.7345122515025427

### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
from sklearn.model_selection import GridSearchCV
parameters = {    
    'vect__ngram_range': ((1, 1), (1, 2)),
    'vect__max_df': (0.5, 0.75, 1.0),
    'vect__max_features': (None, 5000, 10000),
    'tfidf__use_idf': (True, False),
    'clf__estimator__n_estimators': [50, 100, 200],
    'clf__estimator__min_samples_split': [2, 3, 4],
    }
cv = GridSearchCV(pipeline, param_grid=parameters, cv=2, n_jobs=-1)



In [12]:
cv.fit(X_train, y_train)
import warnings
warnings.filterwarnings("ignore")

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [45]:
y_pred_c=cv.predict(X_test)
labels=np.unique(y_pred_c)
cm=skm.multilabel_confusion_matrix(y_test, y_pred_c, labels=labels)
print(cm)
print(skm.classification_report(y_test, y_pred_c))
print("Labels:", labels)
#print("Confusion Matrix:\n", confusion_mat)
accuracy=(y_pred_c==y_test).mean()
print("Accuracy:", accuracy)
print(accuracy.min())


[[[ 566 1488]
  [ 227 6371]]

 [[7065  115]
  [ 764  708]]]
              precision    recall  f1-score   support

           0       0.81      0.97      0.88      6598
           1       0.86      0.48      0.62      1472
           2       0.00      0.00      0.00        38
           3       0.75      0.65      0.69      3545
           4       0.66      0.12      0.20       701
           5       0.82      0.17      0.28       446
           6       0.80      0.09      0.16       226
           7       0.40      0.01      0.02       160
           8       0.52      0.06      0.11       267
           9       0.00      0.00      0.00         0
          10       0.83      0.50      0.62       543
          11       0.83      0.69      0.75       965
          12       0.80      0.45      0.58       775
          13       0.81      0.17      0.29       127
          14       0.67      0.05      0.10       191
          15       0.88      0.07      0.12       104
          16       0.

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms

In [17]:

pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
    ])

In [18]:
parameters = {
        'vect__ngram_range': ((1, 1), (1, 2)),
        'vect__max_df': (0.5, 0.75, 1.0),
        'tfidf__use_idf': (True, False),
        'vect__max_features': (None, 5000, 10000),
    }

cvA = GridSearchCV(pipeline, param_grid=parameters, cv=2)

In [19]:
cvA.fit(X_train, y_train)

GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x000002C5F99AF4C0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier()))]),
             param_grid={'tfidf__use_idf': (True, False),
                         'vect__max_df': (0.5, 0.75, 1.0),
                         'vect__max_features': (None, 5000, 10000),
                         'vect__ngram_range': ((1, 1), (1, 2))})

### 9. Test your model
Show the accuracy, precision, and recall of the tuned model. 

In [46]:
y_pred_a_c=cvA.predict(X_test)
labels=np.unique(y_pred_a_c)
cm=skm.multilabel_confusion_matrix(y_test, y_pred_a_c, labels=labels)
print(cm)
print(skm.classification_report(y_test, y_pred_a_c))
print("Labels:", labels)
#print("Confusion Matrix:\n", confusion_mat)
accuracy=(y_pred_a_c==y_test).mean()
print("Accuracy:", accuracy)
print(accuracy.min())


[[[ 675 1379]
  [ 346 6252]]

 [[6938  242]
  [ 690  782]]]
              precision    recall  f1-score   support

           0       0.82      0.95      0.88      6598
           1       0.76      0.53      0.63      1472
           2       0.00      0.00      0.00        38
           3       0.74      0.58      0.65      3545
           4       0.57      0.28      0.37       701
           5       0.66      0.34      0.45       446
           6       0.60      0.17      0.27       226
           7       0.09      0.01      0.02       160
           8       0.54      0.30      0.38       267
           9       0.00      0.00      0.00         0
          10       0.73      0.60      0.66       543
          11       0.81      0.67      0.73       965
          12       0.76      0.51      0.61       775
          13       0.65      0.34      0.45       127
          14       0.44      0.28      0.34       191
          15       0.42      0.05      0.09       104
          16       0.

In [20]:
print('\n Classification Report \n',skm.classification_report(y_test, y_pred))


 Classification Report 
               precision    recall  f1-score   support

           0       0.81      0.97      0.88      6598
           1       0.89      0.41      0.56      1472
           2       0.00      0.00      0.00        38
           3       0.78      0.61      0.69      3545
           4       0.68      0.05      0.09       701
           5       0.75      0.05      0.09       446
           6       0.67      0.01      0.02       226
           7       0.00      0.00      0.00       160
           8       0.80      0.04      0.09       267
           9       0.00      0.00      0.00         0
          10       0.87      0.20      0.33       543
          11       0.87      0.41      0.56       965
          12       0.84      0.28      0.42       775
          13       0.71      0.04      0.07       127
          14       0.83      0.03      0.05       191
          15       0.00      0.00      0.00       104
          16       0.50      0.00      0.01       293
 

### 10. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms

In [23]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(KNeighborsClassifier()))
            ])

In [24]:
parameters = {
        'vect__ngram_range': ((1, 1), (1, 2)),
        'vect__max_df': (0.5, 0.75, 1.0),
        'tfidf__use_idf': (True, False),
        'clf__estimator__weights': ['uniform', 'distance'],
        'clf__estimator__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
        'clf__estimator__n_neighbors' : [5, 25]
    }
cvK = GridSearchCV(pipeline, param_grid=parameters, cv=2)

In [25]:
cvK.fit(X_train, y_train)

GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x000002C5F99AF4C0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=KNeighborsClassifier()))]),
             param_grid={'clf__estimator__algorithm': ['auto', 'ball_tree',
                                                       'kd_tree', 'brute'],
                         'clf__estimator__n_neighbors': [5, 25],
                         'clf__estimator__weights': ['uniform', 'distance'],
                         'tfidf__use_idf': (True, False),
                         'vect__max_df': (0.5, 0.75, 1.0),
                         'vect__ngram_range': ((1, 1), (1, 2))})

### 11. Test your model
Show the accuracy, precision, and recall of the tuned model. 

In [47]:
y_pred_k_a_c=cvK.predict(X_test)
labels=np.unique(y_pred_k_a_c)
cm=skm.multilabel_confusion_matrix(y_test, y_pred_k_a_c, labels=labels)
print(cm)
print(skm.classification_report(y_test, y_pred_k_a_c))
print("Labels:", labels)
#print("Confusion Matrix:\n", confusion_mat)
accuracy=(y_pred_k_a_c==y_test).mean()
print("Accuracy:", accuracy)
print(accuracy.min())


[[[ 183 1871]
  [  77 6521]]

 [[7155   25]
  [1387   85]]]
              precision    recall  f1-score   support

           0       0.78      0.99      0.87      6598
           1       0.77      0.06      0.11      1472
           2       0.00      0.00      0.00        38
           3       0.78      0.03      0.06      3545
           4       0.33      0.00      0.00       701
           5       0.50      0.01      0.01       446
           6       0.00      0.00      0.00       226
           7       0.00      0.00      0.00       160
           8       0.00      0.00      0.00       267
           9       0.00      0.00      0.00         0
          10       0.67      0.03      0.06       543
          11       0.78      0.05      0.09       965
          12       0.86      0.03      0.06       775
          13       0.75      0.02      0.05       127
          14       0.75      0.02      0.03       191
          15       0.00      0.00      0.00       104
          16       0.

In [30]:
print(cv.best_params_)
print(cvA.best_params_)
print(cvK.best_params_)

{'clf__estimator__min_samples_split': 3, 'clf__estimator__n_estimators': 200, 'tfidf__use_idf': True, 'vect__max_df': 0.75, 'vect__max_features': 10000, 'vect__ngram_range': (1, 2)}
{'tfidf__use_idf': True, 'vect__max_df': 0.75, 'vect__max_features': 10000, 'vect__ngram_range': (1, 1)}
{'clf__estimator__algorithm': 'auto', 'clf__estimator__n_neighbors': 5, 'clf__estimator__weights': 'distance', 'tfidf__use_idf': True, 'vect__max_df': 0.5, 'vect__ngram_range': (1, 1)}


### 9. Export your model as a pickle file

In [36]:
import joblib

In [38]:
joblib.dump(cv.best_params_, 'rf.pkl', compress = 1)


['rf.pkl']

In [39]:

joblib.dump(cvK.best_params_, 'kn.pkl', compress = 1)

['kn.pkl']

In [40]:
joblib.dump(cvA.best_params_, 'ada.pkl', compress = 1)

['ada.pkl']